In [1]:
from moccasin import setup_notebook

setup_notebook()

In [3]:
from moccasin.config import get_active_network

active_network = get_active_network()
print(active_network.name)


eth-forked


In [7]:
from boa.contracts.abi.abi_contract import ABIContract
from typing import Tuple
from moccasin.config import get_active_network
import boa

STARTING_ETH_BALANCE = int(1000e18)
STARTING_WETH_BALANCE = int(1e18)
STARTING_USDC_BALANCE = int(100e6)

def _add_eth_balance():
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)
    
def _add_token_balance(usdc, weth):
    print(f"Starting balance of WETH: {weth.balanceOf(boa.env.eoa)}")
    print(f"Starting balance of USDC: {usdc.balanceOf(boa.env.eoa)}")
    weth.deposit(value = STARTING_WETH_BALANCE)
    our_address = boa.env.eoa
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
    usdc.configureMinter(
        our_address, STARTING_USDC_BALANCE
    )
    print("mint the usdc")
    usdc.mint(our_address, STARTING_USDC_BALANCE)
    print(f"Ending balance of WETH: {weth.balanceOf(boa.env.eoa)}")
    print(f"Ending balance of USDC: {usdc.balanceOf(boa.env.eoa)}")

def setup_script() -> Tuple[ABIContract,ABIContract,ABIContract,ABIContract]:
    print("Starting setup script...")
    
    #1. Give ourselves some ETH
    #2. Give ourselves some USDC and WETH
    active_network = get_active_network()
    
    usdc = active_network.manifest_named("usdc")
    weth = active_network.manifest_named("weth")
    
    if active_network.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc, weth)
        
    
def moccasin_main():
    our_address = boa.env.eoa
    print(our_address,"this is the address")
    print("hello")
    setup_script()
    
moccasin_main()

0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266 this is the address
hello
Starting setup script...
Starting balance of WETH: 1000000000000000000
Starting balance of USDC: 100000000
mint the usdc
Ending balance of WETH: 2000000000000000000
Ending balance of USDC: 200000000


In [9]:
usdc = active_network.manifest_named("usdc")
weth = active_network.manifest_named("weth")



In [12]:
usdc.balanceOf(boa.env.eoa)
weth.balanceOf(boa.env.eoa)

2000000000000000000